In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [ ]:
#Carregar a tabela de buscas de produtos:
produtos_df = pd.read_excel(r'Base de Buscas\buscas.xlsx')
display(produtos_df)

In [ ]:
def buscar_precos_google_shopping(navegador, nome, termos_banidos, preco_minimo, preco_maximo):
    #Abrir o site do Google Shopping:
    navegador.get("https://shopping.google.com.br/")
    #Buscar o produto:
    navegador.find_element(By.ID, 'REsRA').send_keys(nome)
    navegador.find_element(By.CLASS_NAME, 'uDgUL').click()
    #Definir preços:
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    #Crias lista de filtros da busca:
    lista_termos_produto = nome.lower().split(" ")
    lista_termos_banidos = termos_banidos.lower().split(" ")
    
    #Lista que será retornada pela função:
    lista_ofertas_google = []

    while True:
        #Esperar até que os resultados da pesquisa estejam disponíveis na página
        WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'sh-dgr__content')))
        time.sleep(1)
        
        #Pegar todos os produtos da página:
        lista_produtos = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__content')
        
        for produto in lista_produtos:
            #Obter os dados dos produtos:
            nome_produto = produto.find_element(By.TAG_NAME, 'h3').text.lower().replace(',','')
            preco_produto = produto.find_element(By.CLASS_NAME, 'a8Pemb').text.replace(' ', '').replace('R$', '').replace('.', '').replace(',', '.')
            
            #Filtrar pelos dados da busca:
            termo_prod_correto = all(termo in nome_produto for termo in lista_termos_produto)
            termo_banido_encontrado = any(termo in nome_produto for termo in lista_termos_banidos)

            if termo_prod_correto and not termo_banido_encontrado:
                if preco_minimo <= float(preco_produto) <= preco_maximo:
                    link_produto = produto.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    link_produto = link_produto.replace('https://www.google.com.br/url?url=', '')
                    lista_ofertas_google.append((nome_produto, preco_produto, link_produto))
        try:
            # Tentar clicar no botão "Mais"
            proxima_pag = navegador.find_element(By.LINK_TEXT, 'Mais')
            proxima_pag.click()
        except:
            # Caso não haja mais botão "Mais", encerrar o loop
            break

    return lista_ofertas_google

def buscar_precos_buscape(navegador, nome, termos_banidos, preco_minimo, preco_maximo):
    #Abrir o site do Buscape:
    navegador.get("https://www.buscape.com.br/")
    #Buscar o produto:
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(nome)
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_SearchIconWrapper__Knh_x').click()
    #Definir preços:
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    #Crias lista de filtros da busca:
    lista_termos_produto = nome.lower().split(" ")
    lista_termos_banidos = termos_banidos.lower().split(" ")

    #Lista que será retornada pela função:
    lista_ofertas_buscape = []

    while True:
        #Esperar até que os resultados da pesquisa estejam disponíveis na página:
        elemento = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')))
        time.sleep(1)

        #Pegar todos os produtos da página:
        lista_produtos = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

        for produto in lista_produtos:
            #Obter os dados dos produtos:
            nome_produto = produto.find_element(By.TAG_NAME, 'h2').text.lower().replace(',','')
            preco_produto = produto.find_element(By.TAG_NAME, 'p').text.replace(' ', '').replace('R$', '').replace('.', '').replace(',', '.')

            #Filtrar pelos dados da busca:
            termo_prod_correto = all(termo in nome_produto for termo in lista_termos_produto)
            termo_banido_encontrado = any(termo in nome_produto for termo in lista_termos_banidos)

            if termo_prod_correto and not termo_banido_encontrado:
                if preco_minimo <= float(preco_produto) <= preco_maximo:
                    link_produto = produto.get_attribute('href')
                    lista_ofertas_buscape.append((nome_produto, preco_produto, link_produto))
        
        try:
            #Localizar e tentar clicar no botão próxima página:
            lista_paginas = navegador.find_element(By.CLASS_NAME, 'Paginator_paginator__j178K')
            paginas = lista_paginas.find_elements(By.TAG_NAME, 'li')
            proxima_pag = paginas[-1]
            proxima_pag.click()
            
        except:
            #Caso não haja mais o botão, encerrar o loop:
            break
        
    return lista_ofertas_buscape